# Azure ML - Datasets

## Loguearse a Azure ML

Como las acciones que vamos a hacer por CLI o a través del SDK de Python necesitan una autentificación, primero vamos a loguearnos en Azure ML

### Login en Azure ML con el CLI de Azure ML

Para logearnos en Azure hacemos

In [ ]:
!az login

Se nos abrirá el navegador para logearnos

### Crear un cliente de Azure ML con el SDK de Python

Primero creamos dos variables con la ID de la suscripción y el grupo de recursos, como estos son datos personales, no los voy a poner aquí. Lo que voy a hacer es incluirlos en un archivo `.env` que no voy a subir a GitHub

```bash
AZURE_SUSCRIPION_ID="xxxxx-xxxx-xxxx-xxxx-xxxxx"
AZURE_ML_RESOURCE_GRPU_ID="xxxxx-xxxx-xxxx-xxxx-xxxxx"
```

Ahora para leerlos primero necesitasos tener instalado `dotenv` que lo hacemos mediante `pip install python-dotenv`

In [1]:
import os
import dotenv

dotenv.load_dotenv()

AZURE_SUSCRIPION_ID = os.getenv("AZURE_SUSCRIPION_ID")
AZURE_ML_RESOURCE_GRPU_ID = os.getenv("AZURE_ML_RESOURCE_GRPU_ID")


Ahora que tenemos estas variables creamos un cliente

In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient(DefaultAzureCredential(), AZURE_SUSCRIPION_ID, AZURE_ML_RESOURCE_GRPU_ID)

## Datasets

A la hora de entrenar un modelo, lo primero que necesitamos es tener un dataset. Así que vamos a ver cómo crearlo en Azure ML.

Pero antes de eso, vamos a ver qué datasets vamos a usar. Vamos a usar dos datasets, por un lado el dataset [Cats-vs-Dogs](https://www.kaggle.com/datasets/shaunthesheep/microsoft-catsvsdogs-dataset) para entrenar un modelo de clasificación de imágenes y el dataset [mteb/amazon_reviews_multi](https://huggingface.co/datasets/mteb/amazon_reviews_multi/viewer/en) para entrenar un modelo de clasificación de texto.

## Storage account

Vamos a crear una cuenta de almacenamiento fuera de Azure ML para que en caso de borrar los recursos de Azure ML, que los datos no se pierdan por si los está usando otro recurso, por si los vamos a queres usar en otro recurso o por si queremos hacer un backup de los datos.

Para crear el `storage account` en Azure ML hay que seguir los siguientes pasos:
 * Ir al portal de Azure
 * Ahora crear un nuevo grupo de recursos con la siguiente configuración:
   * En `Suscription` elegir la suscripción que hemos creado, si es que la has creado, y si no la que tengas
   * En `Resource group` poner un nombre para el grupo de recursos, en mi caso he puesto `rg-storage-account`
   * En `Region` elegir la región que más te convenga, en mi caso he elegido `France Central`
 * Una vez creado el grupo de recursos, vamos a crear un nuevo recurso. Buscamos `Storage account` en el buscador que se nos abre y le damos a `Create`
   * En `Subscription` elegimos la suscripción que hemos creado, si es que la has creado, y si no la que tengas
   * En `Resource group` elegimos el grupo de recursos que hemos creado
   * En `Name` ponemos un nombre para el servicio almacenamiento, en mi caso he puesto `storageaccountmfn`
   * En `Region` elegimos la región que más nos convenga, en mi caso he elegido `France Central`. Pero lo ideal es que sea la misma región que el grupo de recursos
   * En `Rendimiento` elegimos `Standar`
   * En `Redundancia` elegimos `LRS` ya que no vamos a tener aplicaciones reales, esto es solo un post
   * Por último le damos al botón de `Review + create` y si todo está bien le damos al botón de `Create`

Cuando termine aparecerá un botón de `Go to resource` que nos llevará a la página principal del servicio.

### Crear un contenedor

Una vez estamos en el recurso darle a `Blob service` y se nos abrirá la sección de contenedores, creamos uno nuevo con las siguientes características:
 * En `Name` ponemos un nombre, en mi caso he puesto `storage-account-container`
 * Le damos al botón de `Create`

### Vincular el contenedor a Azure ML

Volvemos a `Studio`, vamos a la sección de `Connections`, le damos al botón de `Connect` y elegimos `Azure Blob Storage`. Ahora le damos a `Next` y elegimos el contenedor que hemos creado. Por último le damos a `Create` a continuación:
 * En `Suscription` elegimos nuestra suscripcción
 * En `Resource group` elegimos el grupo de recursos que hemos creado, en mi caso `rg-storage-account`
 * En `Blob container` elegimos el contenedor que hemos creado, en mi caso `storage-account-container`
 * En `Authentication method` elegimos `Credential based`
 * En `Authentication type` elegimos `Account key`
 * En `Azure` volvemos a nuestro `storage account` y en la sección de `Security + networking` le damos a `Access keys` y copiamos la `Key1` y la pegamos en la `Account key` de `Studio`
 * Por último en `Connection name` le ponemos un nombre a la conexión, en mi caso he puesto `stor_acc_connection`

Por último le damos al botón de `Save` para guardar la conexión.

### Subir los datasets al contenedor

Ahora que tenemos el contenedor y lo tenemos vinculado a Azure ML, vamos a subir los datasets que vamos a usar.

Solo vamos a ver cómo hacerlo mediante la CLI, ya que este post se centra en el uso de `Azure ML`

Volvemos a `Azure`, al `Storage account`, seleccionamos `containers` y entramos en el contenedor que hemos creado. Ahora le damos al botón de `Upload` y seleccionamos los datasets que queremos subir.

Subimos la carpeta `PetImages` del dataset `Cats-vs-Dogs` y la carpeta `en` del dataset `amazon_reviews_multi`.

### Visualizar los datos del dataset `amazon rewiews multi`

Ahora que tenemos los datos podemos utilizar herramientas de `Azure ML` para visualizarlos.

Volvemos a `Studio` y seleccionamos `Data` de la zona `Assets`. Le damos a `Create` y hacemos lo siguiente

 * En `Name` ponemos in nombre para el dataset, en mi caso he puesto `amazon-reviews-mult-english-train`
 * En `Description` ponemos una descripción, en mi caso he puesto `Dataset with reviews of Amazon products with starts in english. Train subset`.
 * En `Type` elegimos `File (uri_file)`

Le damos al botón de `Next`. Pasaremos a elegir la fuente de datos, elegimos `From Azure Storage` y volvemos a darle al botón de `Next`

A continuación tenemos que elegir de qué `blob storage` queremos coger los datos. Elegimos el que hemos creado antes, en mi caso `stor_acc_connection`. Le damos al botón de `Next`

Seleccionamos el archivo `train.json` de la carpeta `en` y le damos al botón de `Next`

Cuando se haya revisado todo, por último le damos al botón de `Create`

Se nos ha creado el `Dataset` de `Azure ML`. Como la interfaz nos ha dejado dentro del `Dataset` le damos a la pestaña `Explore` para poder explorarlo

Si seleccionamos la pestaña `Consume` nos aparece un código para poder cargarlo en un `DataFrame` de `Pandas`

### Visualizar los datos del dataset `Pets-vs-Dogs`

Volvemos a meternos en la sección de `Data` de la zona de `Assets` y le damos a `Create`

 * En `Name` ponemos in nombre para el dataset, en mi caso he puesto `cats-vs-dogs`
 * En `Description` ponemos una descripción, en mi caso he puesto `Dataset with images of cats and dogs`
 * En `Type` elegimos `Folder (uri_folder)`

Le damos a `Next`. Pasaremos a elegir la fuente de datos, elegimos `From Azure Storage` y volvemos a darle al botón de `Next`

A continuación tenemos que elegir de qué `blob storage` queremos coger los datos. Elegimos el que hemos creado antes, en mi caso `stor_acc_connection`. Le damos al botón de `Next`

Seleccionamos la carpeta `PetImages` y le damos al botón de `Next`

Cuando se haya revisado todo, por último le damos al botón de `Create`

Se nos ha creado el `Dataset` de `Azure ML`. Como la interfaz nos ha dejado dentro del `Dataset` le damos a la pestaña `Explore` para poder explorarlo

Si seleccionamos la pestaña `Consume` nos aparece un código para poder cargarlo en un `DataFrame` de `Pandas`